In [1]:
import numpy as np
import pandas as pd
from river import evaluate
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sail.transformers.river.preprocessing import StandardScaler
from sail.models.torch.rnn import RNNRegressor

#### Load Data


In [2]:
X = pd.read_csv("../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
learner_gru = RNNRegressor(
    input_units=12,
    output_units=1,
    hidden_units=100,
    n_hidden_layers=3,
    lr=0.001,
    cell_type="GRU",
    verbose=0
)

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", learner_gru),
]
sail_pipeline = SAILPipeline(steps=steps, scoring="R2", verbosity_level=1, verbosity_interval=100)

#### Train Test Split


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Start Incremental Training


In [6]:
y_preds = []
y_true = []
batch_size = 10

for start in range(0, 5000, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_pipeline.partial_fit(X, y)


>>> Epoch: 1 | Samples Seen: 0 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.024s, Batch Size=10, P_Score=0]            



>>> Epoch: 100 | Samples Seen: 990 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=-.245]            



>>> Epoch: 200 | Samples Seen: 1990 -------------------------------------------------------------------------------------


...

SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=0.369]            



>>> Epoch: 400 | Samples Seen: 3990 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=0.465]            



>>> Epoch: 500 | Samples Seen: 4990 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=0.514]            


#### Save SAIL Pipeline


In [7]:
sail_pipeline.save(".")

[2023-09-25 19:41:25:136] - INFO - SAIL (PyTorch) - Model saved successfully.


'./sail_pipeline'

#### Load SAIL Pipeline


In [8]:
sail_new_pipeline = SAILPipeline.load(".")

SAIL Pipeline Progressive Score: 100%========== [Points: 4990/4990, Elapsed:0.0101s, Metric=R2, Batch Size=4990, P_Score=0.514]


In [9]:
y_preds = []
y_true = []
batch_size = 10

for start in range(5001, 10768, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_new_pipeline.partial_fit(X, y)

  epoch    train_loss     dur
-------  ------------  ------
    501       70.9320  0.0012
    502      586.7608  0.0012
    503      130.7252  0.0013
    504      117.8795  0.0012
    505      244.0257  0.0012
    506      159.6151  0.0015
    507      129.4393  0.0015
    508      459.9520  0.0013
    509      115.4775  0.0014
    510      183.6156  0.0018
    511      328.2495  0.0013
    512      352.2633  0.0013
    513      170.5602  0.0011
    514      167.8503  0.0014
    515      185.4431  0.0011
    516      315.9188  0.0018
    517      268.6629  0.0014
    518      260.9680  0.0013
    519      257.5792  0.0013
    520      271.9582  0.0013
    521      615.4316  0.0012
    522      252.1417  0.0013
    523      206.1322  0.0015
    524      396.0060  0.0011
    525      153.1999  0.0013
    526      470.2961  0.0012
    527      232.6331  0.0013
    528       87.4382  0.0012
    529      207.4537  0.0014
    530       40.7682  0.0015
    531      153.7080  0.0015
    532   

SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.003s, Batch Size=10]      

    600      443.7640  0.0026


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=0.55]            

    601      348.9725  0.0017
    602      150.3206  0.0011
    603      317.2322  0.0011
    604      223.5169  0.0013
    605      502.6994  0.0013
    606      138.5976  0.0011
    607      454.4296  0.0011
    608      388.8708  0.0013
    609      268.1564  0.0012
    610       90.3049  0.0014
    611      468.2427  0.0010
    612      534.9092  0.0010
    613       58.8251  0.0012
    614      274.3161  0.0011
    615      357.8184  0.0010
    616      349.5341  0.0012
    617      787.9042  0.0011
    618      148.8415  0.0012
    619      268.3844  0.0012
    620      664.3602  0.0011
    621      369.7575  0.0011
    622      251.5016  0.0011
    623      277.8998  0.0011
    624      108.7697  0.0012
    625      234.2333  0.0011
    626      303.1529  0.0014
    627      597.9280  0.0012
    628      259.2596  0.0013
    629       80.1692  0.0011
    630       79.5434  0.0012
    631      250.0045  0.0010
    632      280.6478  0.0011
    633      498.7519  0.0013
    634   

...

SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.003s, Batch Size=10]      

   1000      424.4919  0.0026


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=0.622]            

   1001      331.5819  0.0013
   1002      209.0488  0.0012
   1003      355.5238  0.0011
   1004      161.6272  0.0013
   1005       33.9827  0.0011
   1006      129.6656  0.0013
   1007      327.0358  0.0010
   1008       97.8086  0.0012
   1009      155.4717  0.0012
   1010      662.3262  0.0011
   1011      169.8103  0.0012
   1012      177.8635  0.0011
   1013      121.1232  0.0011
   1014      112.3965  0.0013
   1015      650.4752  0.0009
   1016      106.8949  0.0012
   1017      285.3104  0.0012
   1018      250.4707  0.0012
   1019      174.1011  0.0013
   1020      311.9372  0.0013
   1021      221.0600  0.0012
   1022      280.2750  0.0012
   1023      237.4093  0.0013
   1024      198.7411  0.0011
   1025      255.5565  0.0011
   1026       72.1644  0.0011
   1027      179.8704  0.0011
   1028      643.7239  0.0014
   1029      362.3438  0.0011
   1030      191.2144  0.0013
   1031       86.5478  0.0014
   1032      729.7314  0.0011
   1033      317.8845  0.0014
   1034   

   1037      306.2672  0.0015
   1038      594.6248  0.0015
   1039      242.4502  0.0013
   1040      506.4570  0.0012
   1041      217.6476  0.0036
   1042      130.7790  0.0022
   1043      142.6466  0.0018
   1044      500.3757  0.0013
   1045      244.4154  0.0011
   1046       41.6832  0.0013
   1047      413.6937  0.0011
   1048       25.6242  0.0014
   1049      140.9920  0.0012
   1050      153.2394  0.0012
   1051      183.5710  0.0012
   1052      147.4456  0.0012
   1053      280.1154  0.0011
   1054      351.7817  0.0016
   1055      107.6402  0.0015
   1056      252.4988  0.0014
   1057      279.8991  0.0014
   1058      306.8587  0.0012
   1059      156.1654  0.0011
   1060      503.7505  0.0013
   1061      165.3148  0.0017
   1062      205.8524  0.0011
   1063       71.7136  0.0013
   1064      149.1614  0.0012
   1065      297.7599  0.0012
   1066      329.0806  0.0014
   1067      402.5633  0.0013
   1068      333.3676  0.0014
   1069      451.2728  0.0013
   1070   

#### Make Prediction on Hold out set


In [10]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_test.shape[0], batch_size):

    end = start + batch_size

    X = X_test.iloc[start:end]
    y = y_test.iloc[start:end]

    preds = sail_new_pipeline.predict(X)
    y_preds.extend(list(preds))
    y_true.extend(list(y))

#### Final Score


In [11]:
sail_new_pipeline.get_progressive_score

0.6285566617151723

In [12]:
sail_new_pipeline._scorer.score(y_true, y_preds)

SAIL Pipeline Score: 100%========== [Points: 4616/4616, Elapsed:0.0089s, Metric=R2, Batch Size=4616, Score=0.717]


0.7169338953897136

In [13]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(100)
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()